In [ ]:
##create address list from excel file
import pandas as pd
import numpy as np
import re

path = ##Add path to excel file containing addresses here

df = pd.read_excel(path, "Longlats")

##address1 == full list with duplicates, in case you need that later
address1 = df['Address'].tolist()

##remove duplicates
address = list(dict.fromkeys(address1))

In [ ]:
##create address list from multiple excel files
import pandas as pd
import numpy as np

full_address_list = []

fn_base = ##insert folder location
fn_list = [] ##list of spreadsheet names containing addresses

for i in fn_list:
    df = pd.read_excel(fn_base+i)

    ##address1 == full list with duplicates, in case you need that later
    address1 = df['Address'].tolist()
    full_address_list.append(address1)

##flatten list
full_address_list = [item for sublist in full_address_list for item in sublist]

##remove duplicates
address = list(dict.fromkeys(full_address_list))

In [ ]:
##for checking all addresses were read and duplicates were cleaned successfully
len(address)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import time

    ##optional - use an ublock origin on the selenium window
# path_to_extension = '1.30.6_0' ##path to ublock extension file
# chrome_options = Options()
# chrome_options.add_argument('load-extension=' + path_to_extension)

chromedriver = ##path to chrome driver
browser = webdriver.Chrome(executable_path=chromedriver)
# browser.create_options() ##optional - for ublock

url =' https://www.google.com/maps/'
browser.get(url)

In [ ]:
##plus code version - finds and searches the plus code in the location information, then converts the extended plus code from the url to longitude and latitude

longlatlist = []

def geocode(address):
    time.sleep(1)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchboxinput"]'))).send_keys(address)
    time.sleep(1)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchbox-searchbutton"]'))).click()

    ##get short plus code
    time.sleep(2)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[8]')))
    html = browser.page_source
    soup = BeautifulSoup(html, "html.parser")
    aria_label = soup.findAll(attrs={'aria-label': re.compile('\w{4}\+\w{2}\s(.*)')})
    plus_code = re.findall('aria-label="(.*?)"', str(aria_label))
    plus_code = [i.replace('Plus code: ', '') for i in plus_code]
    time.sleep(1)

    ##search short plus code
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchbox"]/a'))).click()
    time.sleep(1)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchboxinput"]'))).send_keys(plus_code[0])
    time.sleep(1)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchbox-searchbutton"]'))).click()

    ##get full plus code
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[8]')))
    current_url = browser.current_url
    full_p_code = re.findall('/place/(.*?)/', current_url)
    full_p_code = full_p_code[0]
    full_p_code = full_p_code.replace('%2B', "")

    ##conversion
    p_code_list = [full_p_code[i:i+1] for i in range(0, len(full_p_code), 1)]
    plus_code_conversion = ['2','3','4','5','6','7','8','9','C','F','G','H','J','M','P','Q','R',"V",'W',"X"]
    con_p_code_list = []
    for i in p_code_list:
        con_p_code_list.append(plus_code_conversion.index(i))
    con_p_code_list = [float(i) for i in con_p_code_list]
    multiplications = [20,20,1,1,0.05,0.05,0.0025,0.0025,0.000125,0.000125,0.000030125,0.000030125]
    converted_p = []
    for num1, num2 in zip(con_p_code_list, multiplications):
        converted_p.append(num1 * num2)
    lon = sum(converted_p[1::2]) - 180
    lat = sum(converted_p[0::2]) - 90

    ##output
    longlatlist.append([lat,lon])

In [ ]:
##version using url only, not plus code reliant - discovered later that the coordinates appear in the url itself, and is much more effective than the plus code method
longlatlist = []

def geocode(address):
    time.sleep(1)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchboxinput"]'))).send_keys(address)
    time.sleep(1)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchbox-searchbutton"]'))).click()

    ##get long lat
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="pane"]/div/div[1]/div/div/div[8]')))
    current_url = browser.current_url
    longlat = current_url[-25:]
    lat = longlat.split("!4d")[0]
    lon = longlat.split("!4d")[1]
    lat = lat.split('-')[1]
    lat = '-'+lat

    ##output
    longlatlist.append([lat,lon])

    time.sleep(2) ##to slow it down a bit just in case

In [ ]:
for i in address:
    try:
        geocode(i)
        try: 
            WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchbox"]/a'))).click()
            ##small wait before continuing
            time.sleep(2)
        except Exception:
            continue
    except Exception:
        longlatlist.append(["",""])
        try: 
            WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchbox"]/a'))).click()
            ##small wait before continuing
            time.sleep(2)
        except Exception:
            continue
        continue

browser.close()

##combine searched address and resulting long lats in dictionary
list_dict = dict(zip(address, longlatlist))

list_dict

In [ ]:
# see dictionary entries (addresses) where coordinates could not be found (value is empty list) - i.e. geocoding failed
example = ["", ""]
empty_dict = {k: v for k, v in list_dict.items() if v == example}
empty_dict

In [ ]:
# see % of dictionary entries where coordinates could not be found
len(empty_dict.keys())/len(list_dict.keys())

In [ ]:
##Dataframe saving in case direct excel does not work
df1 = pd.DataFrame.from_dict(list_dict,orient='columns').transpose()

path2 = ##saving file path

df1.to_excel(path2)

In [ ]:
##update an existing excel workbook with long lats
from openpyxl import load_workbook
import openpyxl

book = load_workbook() ##add workbook load name here
sheet0 = book["Sheet1"]
position = 1
for i in address1:
    key = sheet0.cell(column=1, row=position+1).value
    sheet0.cell(column=6, row=position+1, value=list_dict[key][0])
    sheet0.cell(column=7, row=position+1, value=list_dict[key][1])
    position+=1
book.save() ##add workbook save name here 
book.close()